In [125]:
# !pip install torch torchvision
# !pip install matplotlib pandas scikit-learn
# !pip install pillow
# !pip install opencv-python

In [93]:
dataset_dir = "insect-dataset/moth"

In [104]:
import torch
torch.cuda.is_available()

True

In [ ]:
import shutil
import time
import datetime
import random
import numpy as np
from pathlib import Path
from PIL import Image
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def split_data_for_train_and_val(data_dir, test_dir, val_dir, train_dir, test_data_weight, val_data_weight, min_file_cnt_for_val):
    train_data_cnt = 0
    val_data_cnt = 0
    test_data_cnt = 0
    class_cnt = 0
    
    for class_dir in Path(data_dir).iterdir():
        if class_dir.is_dir() and os.listdir(class_dir):
            class_cnt = class_cnt + 1
            file_count = sum(1 for file in class_dir.iterdir() if file.is_file())
            for file in Path(class_dir).iterdir():
                if file.is_file():
                    random_float = random.random()
                    class_dir_name = class_dir.name
                    if file_count >= min_file_cnt_for_val and random_float < test_data_weight:
                        target_dir = test_dir
                        test_data_cnt = test_data_cnt + 1
                    elif file_count >= min_file_cnt_for_val and random_float < test_data_weight + val_data_weight:
                        target_dir = val_dir
                        val_data_cnt = val_data_cnt + 1
                    else:
                        target_dir = train_dir
                        train_data_cnt = train_data_cnt + 1
                    target_dir_path = f"{target_dir}/{class_dir_name}"
                    if not os.path.exists(target_dir_path):
                        os.makedirs(target_dir_path)
                    shutil.copy(file, target_dir_path)

    print(f"Class count: {class_cnt}")
    print(f"Training data count: {train_data_cnt}")
    print(f"Validation data count: {val_data_cnt}")
    print(f"Test data count: {test_data_cnt}")

In [ ]:
def init_model_for_training(train_dir, val_dir, batch_size):
    transform = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
        'val': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
    }
    training_datasets = {
        'train': datasets.ImageFolder(root=train_dir, transform=transform['train']),
        'val': datasets.ImageFolder(root=val_dir, transform=transform['val']),
    }
    dataloaders = {
        'train': DataLoader(training_datasets['train'], batch_size=batch_size, shuffle=True),
        'val': DataLoader(training_datasets['val'], batch_size=batch_size, shuffle=False),
    }
    class_names = training_datasets['train'].classes
    
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    num_classes = len(class_names)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    return {
        'model': model,
        'device': device,
        'transform': transform,
        'datasets': training_datasets,
        'dataloaders': dataloaders,
        'class_names': class_names,
        'num_classes': num_classes,
        'num_features': num_features,
        'criterion': criterion,
        'optimizer': optimizer,
        'scheduler': scheduler
    }

In [ ]:
def train(model_data, num_epochs, model_path):
    start_time = time.time()
    for epoch in range(num_epochs):
        print(f"Epoch {(epoch+1):4} / {num_epochs:4}", end=' ')
        for phase in ['train', 'val']:
            if phase == 'train':
                model_data['model'].train()
            else:
                model_data['model'].eval()
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in model_data['dataloaders'][phase]:
                inputs, labels = inputs.to(model_data['device']), labels.to(model_data['device'])
                model_data['optimizer'].zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model_data['model'](inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = model_data['criterion'](outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        model_data['optimizer'].step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
    
            epoch_loss = running_loss / len(model_data['datasets'][phase])
            epoch_acc = running_corrects.double() / len(model_data['datasets'][phase])
            print(f" | {phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}", end=' ')
            if phase == 'train':
                model_data['scheduler'].step()
        print(f" | Elapsed time: {datetime.timedelta(seconds=(time.time() - start_time))}")
        torch.save(model_data['model'].state_dict(), model_path)

In [101]:
def predict(image_path, model_data):
    image = Image.open(image_path).convert("RGB")
    image = model_data['transform']['val'](image).unsqueeze(0).to(model_data['device'])
    with torch.no_grad():
        outputs = model_data['model'](image)
        _, preds = torch.max(outputs, 1)
    try:
        return model_data['class_names'][preds[0]]
    except (Exception):
        return None

def predict_top_k(image_path, model_data, k):
    image = Image.open(image_path).convert("RGB")
    image = model_data['transform']['val'](image).unsqueeze(0).to(model_data['device'])
    with torch.no_grad():
        outputs = model_data['model'](image)
        probabilities = F.softmax(outputs, dim=1)
        top_probs, top_indices = torch.topk(probabilities, k)
    try:
        return {model_data['class_names'][top_indices[0][i]]: top_probs[0][i].item() for i in range(0, k)}
    except (Exception):
        return None

# Manually create phases(egg/larva/pupa/moth) dataset 

# Split phases dataset for train/val

In [36]:
if os.path.exists(f"{dataset_dir}/phases-train"):
    shutil.rmtree(f"{dataset_dir}/phases-train")
if os.path.exists(f"{dataset_dir}/phases-val"):
    shutil.rmtree(f"{dataset_dir}/phases-val")
    
split_data_for_train_and_val(f"{dataset_dir}/phases-data", None, f"{dataset_dir}/phases-val", f"{dataset_dir}/phases-train", 0, 0.2, 4)

Class count: 4
Training data count: 1899
Validation data count: 499
Test data count: 0


# Train model to seperate phases

In [11]:
phases_model_data = init_model_for_training(f'{dataset_dir}/phases-train', f'{dataset_dir}/phases-val', 32)

In [42]:
train(phases_model_data, 25, f"{dataset_dir}/phases_checkpoint_latest.pth")

Epoch    1 /   25  | Train Loss: 0.0490 Acc: 0.9821  | Val Loss: 8.5694 Acc: 0.0180  | Elapsed time: 0:00:13.910416
Epoch    2 /   25  | Train Loss: 0.0479 Acc: 0.9842  | Val Loss: 8.1367 Acc: 0.0220  | Elapsed time: 0:00:28.820473
Epoch    3 /   25  | Train Loss: 0.0438 Acc: 0.9863  | Val Loss: 8.4808 Acc: 0.0180  | Elapsed time: 0:00:44.175050
Epoch    4 /   25  | Train Loss: 0.0448 Acc: 0.9831  | Val Loss: 8.3880 Acc: 0.0200  | Elapsed time: 0:00:59.713113
Epoch    5 /   25  | Train Loss: 0.0391 Acc: 0.9884  | Val Loss: 8.6596 Acc: 0.0180  | Elapsed time: 0:01:15.259200
Epoch    6 /   25  | Train Loss: 0.0440 Acc: 0.9847  | Val Loss: 8.3532 Acc: 0.0180  | Elapsed time: 0:01:30.669789
Epoch    7 /   25  | Train Loss: 0.0397 Acc: 0.9863  | Val Loss: 8.7578 Acc: 0.0180  | Elapsed time: 0:01:46.244852
Epoch    8 /   25  | Train Loss: 0.0371 Acc: 0.9879  | Val Loss: 8.6134 Acc: 0.0200  | Elapsed time: 0:02:02.156901
Epoch    9 /   25  | Train Loss: 0.0434 Acc: 0.9858  | Val Loss: 8.5128 

# Discard egg/larva/pupa data using phases model

In [82]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math

def plot_predictions(predictions, cols):
    n_images = len(predictions)
    rows = math.ceil(n_images / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
    axes = axes.flatten()
    for i, prediction in enumerate(predictions):
        image = mpimg.imread(prediction['file'])
        axes[i].imshow(image)
        axes[i].set_title(prediction['prediction'], fontsize=30)
        axes[i].axis('off')
    for i in range(len(predictions), len(axes)):
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

In [12]:
from pathlib import Path

clean_dataset_dir = f"{dataset_dir}/clean-data"
discarded_dataset_dir = f"{dataset_dir}/discarded-data"
min_data_cnt_to_retain_all = 5

if os.path.exists(clean_dataset_dir):
    shutil.rmtree(clean_dataset_dir)
if os.path.exists(discarded_dataset_dir):
    shutil.rmtree(discarded_dataset_dir)

def discard_unwanted_data(data_dir, limit=-1):
    predictions = []
    moth_class_cnt = 0
    larva_class_cnt = 0
    for species_dir in Path(data_dir).iterdir():
        if species_dir.is_dir():
            file_count = sum(1 for file in species_dir.iterdir() if file.is_file())
            for file in Path(f"{species_dir}").iterdir():
                if file.is_file():
                    prediction = predict(file, phases_model_data)
                    species_name = str(file.resolve().as_posix()).split("/")[-2]
                    if prediction == 'moth' or file_count < min_data_cnt_to_retain_all:
                        target_dir = f"{clean_dataset_dir}/{species_name}"
                        if not os.path.exists(target_dir):
                            moth_class_cnt = moth_class_cnt + 1
                            os.makedirs(target_dir)
                    else:
                        target_dir = f"{discarded_dataset_dir}/{species_name}"
                        if not os.path.exists(target_dir):
                            larva_class_cnt = larva_class_cnt + 1
                            os.makedirs(target_dir)
                    shutil.copy(file, target_dir)
                    predictions.append({
                        'file': file,
                        'prediction': prediction
                    })
                    limit = limit - 1
                    if(limit == 0): return predictions
    # print(f"{larva_class_cnt} discarded & {moth_class_cnt} retained")
    return predictions

start_time = time.time()
phases_model_data['model'].load_state_dict(torch.load(f"{dataset_dir}/phases_checkpoint_latest.pth", weights_only=False))
phases_model_data['model'].eval()
predictions = discard_unwanted_data(f'{dataset_dir}/data')

print(f"Elapsed time: {datetime.timedelta(seconds=(time.time() - start_time))}")

# plot_predictions([p for p in predictions if p['prediction'] != 'moth'][:30], 6)

Elapsed time: 0:13:43.031402


# Split dataset for train/val/test

In [13]:
import shutil
from pathlib import Path
import random
import os
import time
import datetime

if os.path.exists(f"{dataset_dir}/test"):
    shutil.rmtree(f"{dataset_dir}/test")
if os.path.exists(f"{dataset_dir}/val"):
    shutil.rmtree(f"{dataset_dir}/val")
if os.path.exists(f"{dataset_dir}/train"):
    shutil.rmtree(f"{dataset_dir}/train")

start_time = time.time()
split_data_for_train_and_val(f"{dataset_dir}/clean-data", f"{dataset_dir}/test", f"{dataset_dir}/val", f"{dataset_dir}/train", 0.05, 0.15, 4)
print(f"Elapsed time: {datetime.timedelta(seconds=(time.time() - start_time))}")

Class count: 3048
Training data count: 34618
Validation data count: 6085
Test data count: 2105
Elapsed time: 0:02:43.535177


# Train model

In [14]:
model_data = init_model_for_training(f'{dataset_dir}/train', f'{dataset_dir}/val', 32)

In [19]:
train(model_data, 25, f"{dataset_dir}/checkpoint_latest.pth")

Epoch    1 /   25  | Train Loss: 3.9159 Acc: 0.2084  | Val Loss: 18.0966 Acc: 0.0005  | Elapsed time: 0:04:44.085452
Epoch    2 /   25  | Train Loss: 2.8968 Acc: 0.3528  | Val Loss: 19.4684 Acc: 0.0007  | Elapsed time: 0:09:23.433819
Epoch    3 /   25  | Train Loss: 2.6383 Acc: 0.3988  | Val Loss: 20.2678 Acc: 0.0003  | Elapsed time: 0:14:02.541753
Epoch    4 /   25  | Train Loss: 2.4828 Acc: 0.4260  | Val Loss: 20.5795 Acc: 0.0005  | Elapsed time: 0:18:42.919077
Epoch    5 /   25  | Train Loss: 2.3558 Acc: 0.4524  | Val Loss: 21.6312 Acc: 0.0002  | Elapsed time: 0:23:23.834475
Epoch    6 /   25  | Train Loss: 2.2246 Acc: 0.4792  | Val Loss: 21.6839 Acc: 0.0005  | Elapsed time: 0:28:02.378643
Epoch    7 /   25  | Train Loss: 2.1243 Acc: 0.4973  | Val Loss: 22.2613 Acc: 0.0005  | Elapsed time: 0:32:42.767700
Epoch    8 /   25  | Train Loss: 2.0044 Acc: 0.5243  | Val Loss: 23.0975 Acc: 0.0003  | Elapsed time: 0:37:23.766092
Epoch    9 /   25  | Train Loss: 1.8559 Acc: 0.5615  | Val Loss:

# Test model

In [27]:
import pprint

def validate_prediction_in_dir(test_dir, model_data):
    total = 0
    success = 0
    failures = {}
    for species_dir in Path(test_dir).iterdir():
        if species_dir.is_dir():
            for file in Path(f"{species_dir}").iterdir():
                if file.is_file():
                    species = file.parts[-2]
                    prediction = predict(file, model_data)
                    is_success = (species==prediction)
                    if not is_success:
                        failures[species] = prediction
                    total = total + 1
                    if is_success:
                        success = success + 1
    return {
        'total': total, 
        'success': success,
        'failures': failures
    }

def test(checkpoint, test_dir):
    model_data['model'].load_state_dict(torch.load(checkpoint, weights_only=False))
    model_data['model'].eval()
    start_time = time.time()
    prediction = validate_prediction_in_dir(test_dir, model_data)
    print(f"Accuracy: {prediction['success']} / {prediction['total']} -> {100*prediction['success']/prediction['total']:.2f}%")
    print(f"Elapsed time: {datetime.timedelta(seconds=(time.time() - start_time))}")
    print("-"*10)
    print("Failures:")
    pprint.pprint(prediction['failures'])

In [21]:
test(f"{dataset_dir}/checkpoint_latest.pth", f"{dataset_dir}/test")

Accuracy: 814 / 2105 -> 38.67%
Elapsed time: 0:00:16.496665
----------
Failures:
{'Aiteta-spp': 'banisia-spp',
 'Alcis-periphracta': 'dasyboarmia-spp',
 'Apha-floralis': 'banisia-spp',
 'Boeotarcha-martinalis': 'oxaenanus-brontesalis',
 'Calamotropha-corticellus': 'westermannia-superba',
 'Calletaera-obliquata': 'Luxiaria-hypaphanes',
 'Callopistria-pulchrilinea': 'dudusa-sphingiformis',
 'Carea-obsolescens': 'oxymacaria-palliata',
 'Chalcosiopsis-variata': 'Mosopia-kononenkoi',
 'Charitoprepes-lubricosa': 'crambidae-genera-spp',
 'Chloroplaga-pallida': 'Somatina-densifasciaria',
 'Chusaris-spp': 'rhesala-spp',
 'Comibaena-biplaga': 'poliobotys-ablactalis',
 'Cusiala-raptaria': 'Cusiala-spp',
 'Cusiala-spp': 'phalera-spp',
 'Daplasa-irrorata': 'aegocera-venulia',
 'Ecpyrrhorrhoe-spp': 'perina-nuda',
 'Floridasura-tricolor': 'Moorasura-inflexa',
 'Goniocraspidum-ennomoide': 'lasiocampidae-genera-spp',
 'Harutaea-flavizona': 'heterostegane-spp',
 'Hyperythra-swinhoei': 'apona-caschmirens

# Train & Test again

In [22]:
train(model_data, 20, f"{dataset_dir}/checkpoint_latest.pth")

Epoch    1 /   20  | Train Loss: 1.7439 Acc: 0.5875  | Val Loss: 23.1811 Acc: 0.0003  | Elapsed time: 0:04:25.660752
Epoch    2 /   20  | Train Loss: 1.7487 Acc: 0.5862  | Val Loss: 23.3453 Acc: 0.0002  | Elapsed time: 0:09:05.312824
Epoch    3 /   20  | Train Loss: 1.7470 Acc: 0.5880  | Val Loss: 23.2512 Acc: 0.0002  | Elapsed time: 0:13:43.395756
Epoch    4 /   20  | Train Loss: 1.7415 Acc: 0.5876  | Val Loss: 23.2180 Acc: 0.0003  | Elapsed time: 0:18:21.717836
Epoch    5 /   20  | Train Loss: 1.7465 Acc: 0.5901  | Val Loss: 23.3627 Acc: 0.0002  | Elapsed time: 0:23:03.262890
Epoch    6 /   20  | Train Loss: 1.7485 Acc: 0.5852  | Val Loss: 23.3329 Acc: 0.0002  | Elapsed time: 0:27:42.845197
Epoch    7 /   20  | Train Loss: 1.7438 Acc: 0.5876  | Val Loss: 23.1874 Acc: 0.0002  | Elapsed time: 0:32:23.790829
Epoch    8 /   20  | Train Loss: 1.7379 Acc: 0.5911  | Val Loss: 23.3091 Acc: 0.0002  | Elapsed time: 0:37:07.961059
Epoch    9 /   20  | Train Loss: 1.7421 Acc: 0.5890  | Val Loss:

In [45]:
shutil.copy(f"{dataset_dir}/checkpoint_latest.pth", f"{dataset_dir}/checkpoint_{int(time.time())}.pth")

'insect-dataset/moth/checkpoint_1737807164.pth'

In [28]:
test(f"{dataset_dir}/checkpoint_latest.pth", f"{dataset_dir}/test")

Accuracy: 804 / 2105 -> 38.19%
Elapsed time: 0:00:15.555013
----------
Failures:
{'Aiteta-spp': 'banisia-spp',
 'Alcis-periphracta': 'dasyboarmia-spp',
 'Apha-floralis': 'Gunda-spp',
 'Boeotarcha-martinalis': 'oxaenanus-brontesalis',
 'Calamotropha-corticellus': 'westermannia-superba',
 'Calletaera-obliquata': 'Luxiaria-hypaphanes',
 'Callopistria-pulchrilinea': 'dudusa-sphingiformis',
 'Carea-obsolescens': 'oxymacaria-palliata',
 'Chalcosiopsis-variata': 'Mosopia-kononenkoi',
 'Charitoprepes-lubricosa': 'crambidae-genera-spp',
 'Chloroplaga-pallida': 'Somatina-densifasciaria',
 'Chusaris-spp': 'rhesala-spp',
 'Comibaena-biplaga': 'poliobotys-ablactalis',
 'Cusiala-raptaria': 'cleora-alienaria',
 'Cusiala-spp': 'phalera-spp',
 'Daplasa-irrorata': 'aegocera-venulia',
 'Ecpyrrhorrhoe-spp': 'perina-nuda',
 'Floridasura-tricolor': 'Moorasura-inflexa',
 'Goniocraspidum-ennomoide': 'lasiocampidae-genera-spp',
 'Harutaea-flavizona': 'heterostegane-spp',
 'Hyperythra-swinhoei': 'herpetogramma-

In [80]:
predict_top_k(Path(f"{dataset_dir}/test_000/antheraea-paphia.jpg"), model_data, 5)

{'antheraea-paphia': 0.3467530608177185,
 'tibetanja-tagoroides': 0.12467645108699799,
 'gastropacha-spp': 0.11187088489532471,
 'hyposidra-talaca': 0.07221446186304092,
 'cricula-trifenestrata': 0.04722161963582039}

In [81]:
predict_top_k(Path(f"{dataset_dir}/test_000/angonyx-krishna.jpg"), model_data, 5)

{'ambulyx-moorei': 0.09101460874080658,
 'deilephila-elpenor': 0.06724270433187485,
 'theretra-alecto': 0.05991685017943382,
 'theretra-clotho': 0.0594797246158123,
 'parasa-lepida': 0.05015980079770088}

In [82]:
predict_top_k(Path(f"{dataset_dir}/test_000/angonyx-krishna-2.jpg"), model_data, 5)

{'angonyx-krishna': 0.5652188658714294,
 'parasa-lepida': 0.11934609711170197,
 'nephele-hespera': 0.08489922434091568,
 'mocis-frugalis': 0.06565359234809875,
 'theretra-clotho': 0.037341922521591187}

# Again

In [86]:
if os.path.exists(f"{dataset_dir}/test_001"):
    shutil.rmtree(f"{dataset_dir}/test_001")
if os.path.exists(f"{dataset_dir}/val_001"):
    shutil.rmtree(f"{dataset_dir}/val_001")
if os.path.exists(f"{dataset_dir}/train_001"):
    shutil.rmtree(f"{dataset_dir}/train_001")
    
split_data_for_train_and_val(f"{dataset_dir}/clean-data", f"{dataset_dir}/test_001", f"{dataset_dir}/val_001", f"{dataset_dir}/train_001", 0.1, 0.2, 4)

Class count: 3048
Training data count: 30594
Validation data count: 8168
Test data count: 4046


In [92]:
model_data = init_model_for_training(f'{dataset_dir}/train_001', f'{dataset_dir}/val_001', 32)
model_data['model'].fc = torch.nn.Linear(512, 3047)
model_data['model'] = model_data['model'].to("cuda:0")
model_data['model'].load_state_dict(torch.load(f"{dataset_dir}/checkpoint_latest.pth", weights_only=False))
train(model_data, 10, f"{dataset_dir}/checkpoint_latest.pth")
shutil.copy(f"{dataset_dir}/checkpoint_latest.pth", f"{dataset_dir}/checkpoint_{int(time.time())}.pth")

Epoch    1 /   10  | Train Loss: 7.8600 Acc: 0.0264  | Val Loss: 8.9647 Acc: 0.0004  | Elapsed time: 0:04:12.757773
Epoch    2 /   10  | Train Loss: 7.2571 Acc: 0.0568  | Val Loss: 9.3002 Acc: 0.0002  | Elapsed time: 0:08:29.751477
Epoch    3 /   10  | Train Loss: 6.9978 Acc: 0.0824  | Val Loss: 10.1264 Acc: 0.0001  | Elapsed time: 0:12:46.166905
Epoch    4 /   10  | Train Loss: 6.7195 Acc: 0.1084  | Val Loss: 10.0348 Acc: 0.0001  | Elapsed time: 0:17:05.766331
Epoch    5 /   10  | Train Loss: 6.4070 Acc: 0.1428  | Val Loss: 10.7305 Acc: 0.0004  | Elapsed time: 0:21:22.545590
Epoch    6 /   10  | Train Loss: 6.1008 Acc: 0.1749  | Val Loss: 11.0124 Acc: 0.0000  | Elapsed time: 0:25:59.090695
Epoch    7 /   10  | Train Loss: 5.8054 Acc: 0.2071  | Val Loss: 11.8888 Acc: 0.0000  | Elapsed time: 0:30:34.954839
Epoch    8 /   10  | Train Loss: 5.2650 Acc: 0.2680  | Val Loss: 12.3435 Acc: 0.0000  | Elapsed time: 0:34:53.345911
Epoch    9 /   10  | Train Loss: 5.1117 Acc: 0.2837  | Val Loss: 1

'insect-dataset/moth/checkpoint_1737811902.pth'

In [97]:
test(f"{dataset_dir}/checkpoint_latest.pth", f"{dataset_dir}/test")

Accuracy: 643 / 2105 -> 30.55%
Elapsed time: 0:00:15.620167
----------
Failures:
{'Aiteta-spp': 'plecoptera-spp',
 'Alcis-periphracta': 'alcis-perspicuata',
 'Boeotarcha-martinalis': 'ozarba-bipars',
 'Calamotropha-corticellus': 'acontia-marmoralis',
 'Calletaera-obliquata': 'Luxiaria-hypaphanes',
 'Callopistria-pulchrilinea': 'areas-galactina',
 'Carea-obsolescens': 'Luxiaria-hypaphanes',
 'Chalcosiopsis-variata': 'Macrobrochis-flavicincta',
 'Charitoprepes-lubricosa': 'hypertrocta-posticalis',
 'Chloroplaga-pallida': 'Chalcosiopsis-variata',
 'Chusaris-spp': 'stenoloba-glaucescens',
 'Comibaena-biplaga': 'anticarsia-irrorata',
 'Cusiala-raptaria': 'cleora-spp',
 'Cusiala-spp': 'idaea-violacea',
 'Daplasa-irrorata': 'speiredonia-mutabilis',
 'Endotricha-luteogrisalis': 'syntypistis-pallidifascia',
 'Eugraptoblemma-rosealis': 'callabraxas-amanda',
 'Floridasura-tricolor': 'barsine-group-spp',
 'Goniocraspidum-ennomoide': 'nola-internella-analis-complex',
 'Harutaea-flavizona': 'biston-

In [100]:
test(f"{dataset_dir}/checkpoint_latest.pth", f"{dataset_dir}/test_001")

Accuracy: 983 / 4046 -> 24.30%
Elapsed time: 0:00:52.197689
----------
Failures:
{'Acygnatha-nigripuncta': 'areas-galactina',
 'Addaea-trimeronalis': 'nola-taeniata',
 'Aeolanthes-rhodochrysa': 'macroglossum-sitiene',
 'Alcis-periphracta': 'alcis-variegata',
 'Alophogaster-rubribasis': 'pidorus-glaucopis',
 'Borbotana-nivifascia': 'erpis-macularis',
 'Calamotropha-spp': 'autocharis-amethystina',
 'Calletaera-obliquata': 'Luxiaria-hypaphanes',
 'Callopistria-pulchrilinea': 'areas-galactina',
 'Cassephyra-lamprosticta': 'boarmiini-genera-spp',
 'Chalcophaedra-zuleika': 'campylotes-histrionicus',
 'Chalcosiopsis-variata': 'Macrobrochis-flavicincta',
 'Chorodna-creataria': 'zythos-avellanea',
 'Chusaris-spp': 'naarda-spp',
 'Comibaena-biplaga': 'Rivula-simulatrix',
 'Cusiala-spp': 'chiasmia-xanthonora',
 'Cyclosiella-dulcicula': 'hyperythra-lutea',
 'Cystidia-indrasana': 'omiodes-milvinalis',
 'Dichromia-indicatalis': 'Tarsolepis-malayana',
 'Ecpyrrhorrhoe-damastesalis': 'amblychia-angeron

# No... Its not working... 24% is too low

### Problems identified
1. not having correct train/eval mode before operation gives rong results
2. model need to be saved with rest of the parameters, at least class to index mapping
3. new class names needs to appended correctly before retraining same model with more classes 